In [64]:
import pandas as pd 
import numpy as np
import re
import os

folder_path= './PassengerVehicle_Stats'
dfs=[]
files= [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path,file)
    df1 = pd.read_csv(file_path)
    dfs.append(df1)

combine_df = pd.concat(dfs, ignore_index=True)
combine_df.to_csv('./vehicles_df.csv', index= False)

In [65]:
df = pd.read_csv('vehicles_df.csv')
df.isna().sum()

Unnamed: 0                                0
Public Vehicle Number                     0
Status                                    0
Vehicle Make                           7668
Vehicle Model                          7852
Vehicle Model Year                     7768
Vehicle Color                          7944
Vehicle Fuel Source                       0
Wheelchair Accessible                     0
Company Name                              0
Address                                7144
City                                   7144
State                                  7144
ZIP Code                               7144
Taxi Affiliation                      37016
Taxi Medallion License Management     37124
Record ID                                 0
dtype: int64

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66408 entries, 0 to 66407
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          66408 non-null  int64  
 1   Public Vehicle Number               66408 non-null  int64  
 2   Status                              66408 non-null  object 
 3   Vehicle Make                        58740 non-null  object 
 4   Vehicle Model                       58556 non-null  object 
 5   Vehicle Model Year                  58640 non-null  float64
 6   Vehicle Color                       58464 non-null  object 
 7   Vehicle Fuel Source                 66408 non-null  object 
 8   Wheelchair Accessible               66408 non-null  object 
 9   Company Name                        66408 non-null  object 
 10  Address                             59264 non-null  object 
 11  City                                59264

In [67]:
(df['Wheelchair Accessible']=='Y').sum()

2496

In [68]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [69]:
(df['Vehicle Model Year'].apply(lambda x: isinstance(x,float))).sum()

66408

In [70]:
df['Vehicle Model Year'].unique()

array([2014., 2010., 2008., 2015., 2012., 2016., 2011., 2017., 2009.,
       2013., 2024., 2005., 2001., 2002., 2000., 1993., 2007., 1998.,
       2006., 1997., 1988., 1990., 2004., 1999., 1986., 1992., 2021.,
       2020., 2023., 2018., 2019., 2022.,   nan, 2003., 2025., 1996.,
       1984., 1994., 1981., 1995., 1991., 1989., 1983., 1987., 1985.,
          7., 1980., 1196.,    0.])

In [71]:

values_replace = [7.,0.]
df['Vehicle Model Year']=df['Vehicle Model Year'].replace(values_replace,np.nan)


In [72]:
df['Vehicle Model Year']=df['Vehicle Model Year'].fillna(0)

In [73]:
df['Vehicle Model Year']=df['Vehicle Model Year'].astype(int)

In [74]:
df.tail()

,Public Vehicle Number,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,Address,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID
66403,42,ACTIVE,PRECISION,6 SEATER,2020,FUSCIA,Pedal,N,JBL PEDICAB MANAGEMENT LLC,1627 N. TRIPP AVE.,CHICAGO,IL,60639.0,NaN,NaN,42Pedicab
66404,52,ACTIVE,SCHWINN,NONE,2014,GREEN,Pedal,N,BM PEDICAB INC.,4450 N. KEDZIE AVE.,CHICAGO,IL,60625.0,NaN,NaN,52Pedicab
66405,70,VIOLATION,MAINSTREET,CLASSIC,0,RED/BLACK,Pedal,N,EMPIRE PEDICAB LLC,318 W. ADAMS ST.,CHICAGO,IL,60606.0,NaN,NaN,70Pedicab
66406,57,VIOLATION,MAINSTREET,BOARDWALK,2011,RED,Pedal,N,CARGILL C KELLY II,2428 W. 24TH PL.,CHICAGO,IL,60608.0,NaN,NaN,57Pedicab
66407,117,ACTIVE,SCHWINN,NONE,2014,GREEN,Pedal,N,BM PEDICAB INC.,4450 N. KEDZIE AVE.,CHICAGO,IL,60625.0,NaN,NaN,117Pedicab


In [75]:
df['Vehicle Make'].unique()

array(['CHEVROLET', 'MERCEDES', 'VAN HOOL', 'VPG', 'DODGE', 'TOYOTA',
       'FORD', 'CADILLAC', 'FREIGHLINE', 'CHAMPION', 'THOMAS',
       'FREIGHTLINER', 'MCI', nan, 'ALEXANDER DENNIS LTD', 'GILLIG',
       'INTL', 'LEYLAND', 'ALEXDER DENNIS LTD', 'IC', 'DOUBLE SHUFLE',
       'ISUZU', 'SPEC-CONSTED', 'TESLA', 'KIA', 'TUK TUK', 'MAINSTREET',
       'LUCID', 'POLARIS', 'BMW', 'PRECISION', 'CHRYSLER', 'MERCURY',
       'LINCOLN', 'GMC', 'ALEXANDER', 'VOLVO', 'HONDA', 'HYUNDAI',
       'NISSAN', 'AUDI', 'LEXUS', 'INFINITI', 'JEEP', 'MOVITRON', 'ORION',
       'ANKAI', 'MASERATI', 'MAZDA', 'CHARLESTON', 'NEOPLAN', 'GENISIS',
       'OSHKOSH', 'BLUEBIRD', 'VOLKSWAGEN', 'BOYERTOWN', 'MITSUBISHI',
       'MOBILITY VENTURE', 'LAND ROVER', 'TURTLE TOP', 'SUBARU', 'ACURA',
       'SCION', 'Dodge', 'Ford', 'PONTIAC', 'BMX', 'BARTH', 'JAGUAR',
       'CINDERELLA', 'TROYER', 'KIMBALL', 'YODER', 'MARTIN', 'SCHWARTZ',
       'MILLER', 'ANDERSON', 'SCHROCK', 'VICTORIA', 'STUDEBAKER',
       'MANMADE

In [76]:
df['ZIP Code']=df['ZIP Code'].fillna(0)

In [77]:
df['ZIP Code']=df['ZIP Code'].astype(int)

In [78]:
df.drop_duplicates(subset=['Public Vehicle Number'],inplace=True)

In [79]:
df.drop(columns=['Address'],inplace=True)

In [80]:
df.drop(columns=['Public Vehicle Number'],inplace=True)

In [81]:
public_column = df['Record ID']

In [82]:
public_column

0        12009Charter Sightseeing
1        12248Charter Sightseeing
2        13527Charter Sightseeing
4        13528Charter Sightseeing
5        12025Charter Sightseeing
                   ...           
16409                     998Taxi
16410                    3810Taxi
16411                    4690Taxi
16412                    2831Taxi
16413                    1509Taxi
Name: Record ID, Length: 14473, dtype: object

In [83]:
df['Vehicle Type'] = df['Record ID'].str.extract(r'([a-zA-Z\s]+)$')

In [84]:
df['Taxi Affiliation']=df['Taxi Affiliation'].fillna('none')

In [85]:
df['Taxi Medallion License Management ']=df['Taxi Medallion License Management '].fillna('none')

In [86]:
df['State'].isna().sum()

1688

In [87]:
df.tail()

,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID,Vehicle Type
16409,VIOLATION,TOYOTA,CAMRY,2012,WHITE,Hybrid,N,AGOMONY CAB INC,CHICAGO,IL,60659,DIS-AFFILIATED,Owner Manager,998Taxi,Taxi
16410,ACTIVE,TOYOTA,RAV4,2017,GRAY/WHITE,Hybrid,N,FENKIL ASMERA TAXI COMPANY,CHICAGO,IL,60613,Choice Taxi Association Inc,Owner Manager,3810Taxi,Taxi
16411,VIOLATION,TOYOTA,CAMRY,2014,WHITE,Hybrid,N,MURIELLE CADICHON,CHICAGO,IL,60643,DIS-AFFILIATED,Owner-Operator,4690Taxi,Taxi
16412,ACTIVE,TOYOTA,PRIUS,2012,WHITE,Hybrid,N,G S TRANS INC,CHICAGO,IL,60659,City Service Taxi Association,Owner Manager,2831Taxi,Taxi
16413,ACTIVE,TOYOTA,SIENNA,2023,WHITE,Hybrid,Y,J A SHALOM LLC,CHICAGO,IL,60612,Independent1,Owner-Operator,1509Taxi,Taxi


In [88]:
Q1 = df['Vehicle Model Year'].quantile(0.25)
Q3 = df['Vehicle Model Year'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Vehicle Model Year'] < lower_bound) | (df['Vehicle Model Year'] > upper_bound)]

print("Outliers:\n", outliers)

Outliers:
          Status Vehicle Make Vehicle Model  Vehicle Model Year Vehicle Color  \
134    INACTIVE     CHAMPION       TROLLEY                1993        ORANGE   
146    INACTIVE      LEYLAND      OLYMPIAN                1988      BURGUNDY   
151    INACTIVE     CHAMPION       TROLLEY                1990         GREEN   
194    INACTIVE      LEYLAND      OLYMPIAN                1988      BURGUNDY   
198    INACTIVE        ISUZU      CARRYALL                1986        MAROON   
...         ...          ...           ...                 ...           ...   
14353  INACTIVE          NaN           NaN                   0         WHITE   
15238  INACTIVE          NaN           NaN                   0         WHITE   
15700  INACTIVE          NaN           NaN                   0         WHITE   
16090  INACTIVE          NaN           NaN                   0         WHITE   
16396  INACTIVE          NaN           NaN                   0         WHITE   

      Vehicle Fuel Source Wh

In [91]:
df['Vehicle Model Year'].value_counts()

Vehicle Model Year
0       1820
2023    1395
2014    1302
2013    1151
2012    1040
2022     993
2019     740
2024     708
2011     705
2015     700
2016     669
2020     556
2021     545
2017     487
2018     453
2010     267
2008     167
2006     154
2007     109
2009     107
2005      64
2001      57
2025      50
2004      43
2002      35
2003      29
1999      27
1997      22
2000      20
1998      13
1996       7
1991       5
1988       5
1992       4
1995       4
1993       3
1984       3
1989       2
1994       2
1986       2
1990       2
1981       1
1983       1
1987       1
1985       1
1980       1
1196       1
Name: count, dtype: int64

In [92]:
df['State'].unique()

array(['IL', nan], dtype=object)

In [96]:
df=df.dropna(subset=['State'])

In [97]:
df['State'].isna().sum()

0

In [98]:
df.shape

(12785, 15)

In [101]:
df['Vehicle Make'].isna().sum()

113

In [102]:
df.to_csv('cleaned_vehicles.csv', index=False)


In [103]:
df2=df.sample(50)

In [105]:
df2.to_csv('cleaned_vehicles_sample.csv', index=False)